### Convert CSV into small csv chunk for the sampling purpose

#### Data-Set
For this assignment purpose, the dataset will be used from [Kaggle Competition](https://www.kaggle.com/c/avazu-ctr-prediction/). Mostlikely, the same challenge will be capstone project as well.



In [1]:
import pandas as pd

df = pd.read_csv('../data/train.csv', nrows=2)
print(df.columns)
df.head()

columns = ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
           'device_ip', 'device_model']

for c in columns:
    df[c] = df[c].apply(lambda x: int(x, 16))
    print(df[c].head(1))

chunk_count = None
chunk_size = 50000

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')
0    532546046
Name: site_id, dtype: int64
0    4085536615
Name: site_domain, dtype: int64
0    680550077
Name: site_category, dtype: int64
0    3970769798
Name: app_id, dtype: int64
0    2013391065
Name: app_domain, dtype: int64
0    131587874
Name: app_category, dtype: int64
0    2845778250
Name: device_id, dtype: int64
0    3721564782
Name: device_ip, dtype: int64
0    1150642724
Name: device_model, dtype: int64


### Converting csv file into sqllite database
Note: The actual function call is commented out so that this function does not get called all the time.

In [5]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
import os

chunk_size = 50000

def process_data_type(df):
    df.hour = df.hour.astype(str)
    df.hour = df.hour.str[:2] + '-' + df.hour.str[2:]
    df.hour = df.hour.str[:5] + '-' + df.hour.str[5:]
    df.hour = df.hour.str[:8] + ' ' + df.hour.str[8:]
    df.hour = pd.to_datetime(df.hour, format='%y-%m-%d %H')
    df.hour = df['hour'].apply(lambda x: x.toordinal())
    
    columns = ['site_id', 'site_domain', 'site_category',
               'app_id', 'app_domain', 'app_category',
               'device_id', 'device_ip', 'device_model']

    for c in columns:
        df[c] = df[c].apply(lambda x: int(x, 16))
    
    return df


def divide_csv_into_chuncks(csv_data):
    start = dt.datetime.now()
    chunksize = chunk_size
    j = 0
    index_start = 1
    dir_path = os.path.dirname(os.path.abspath(csv_data))
    csv_name = os.path.basename(csv_data).split('.')[0]
    for df in pd.read_csv(csv_data, chunksize=chunksize, iterator=True, encoding='utf-8', low_memory=False):
        df = process_data_type(df)
        df.index += index_start
        j+=1
        file_name = os.path.join(dir_path, csv_name + '_chunk%d.csv' % j)
        df.to_csv(file_name, index=False)
        index_start = df.index[-1] + 1
        
        print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
    
    return j


### Check Database Integrity

In [ ]:
#chunk_count = divide_csv_into_chuncks('../data/train.csv')

In [6]:
test_chunk_count = divide_csv_into_chuncks('../data/test.csv')

2 seconds: completed 50000 rows
4 seconds: completed 100000 rows
7 seconds: completed 150000 rows
10 seconds: completed 200000 rows
12 seconds: completed 250000 rows
15 seconds: completed 300000 rows
18 seconds: completed 350000 rows
20 seconds: completed 400000 rows
23 seconds: completed 450000 rows
25 seconds: completed 500000 rows
28 seconds: completed 550000 rows
31 seconds: completed 600000 rows
33 seconds: completed 650000 rows
36 seconds: completed 700000 rows
38 seconds: completed 750000 rows
41 seconds: completed 800000 rows
44 seconds: completed 850000 rows
47 seconds: completed 900000 rows
49 seconds: completed 950000 rows
52 seconds: completed 1000000 rows
55 seconds: completed 1050000 rows
57 seconds: completed 1100000 rows
60 seconds: completed 1150000 rows
63 seconds: completed 1200000 rows
65 seconds: completed 1250000 rows
68 seconds: completed 1300000 rows
71 seconds: completed 1350000 rows
74 seconds: completed 1400000 rows
76 seconds: completed 1450000 rows
79 secon

In [7]:
test_chunk_count

92